In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
from shared_utils import rt_utils, rt_dates, geography_utils

from calitp_data_analysis.tables import tbls
import calitp_data_analysis.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
from rt_analysis import rt_filter_map_plot
# import build_speedmaps_index

from IPython.display import display, Markdown, Latex, HTML
import json

In [2]:
analysis_date = dt.date(2023, 5, 17)

In [3]:
%%capture_parameters
human_date = analysis_date.strftime('%B %d %Y (%A)')
human_date

{"human_date": "May 17 2023 (Wednesday)"}


In [4]:
from tqdm.notebook import tqdm

# Transit on SHS by district (from `bus_service_increase`)

In [32]:
path = 'gs://calitp-analytics-data/data-analyses/bus_service_increase/parallel_or_intersecting_2023-04-12.parquet'

In [33]:
shs_pa = gpd.read_parquet(path)

In [34]:
## probably in feet? (yes)

In [35]:
shs_pa.crs

<Projected CRS: EPSG:2229>
Name: NAD83 / California zone 5 (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - California - counties Kern; Los Angeles; San Bernardino; San Luis Obispo; Santa Barbara; Ventura.
- bounds: (-121.42, 32.76, -114.12, 35.81)
Coordinate Operation:
- name: SPCS83 California zone 5 (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [36]:
## highway lengths are presumably segments of some kind, not full rts

In [37]:
shs_pa >> head(3)

,feed_key,name,route_id,geometry,total_routes,route_length,Route,County,District,RouteType,NB,SB,EB,WB,highway_length,pct_route,pct_highway,parallel
0,026ed9b585ddcd860d59a367139d7429,Bay Area 511 AC Transit Schedule,10,"LINESTRING (5377530.006 3182846.533, 5377574.4...",113,40538.084415,112.0,ALA,4.0,State,0.0,0.0,1.0,1.0,9439.064210,0.130,0.557,0
1,026ed9b585ddcd860d59a367139d7429,Bay Area 511 AC Transit Schedule,10,"LINESTRING (5377530.006 3182846.533, 5377574.4...",113,40538.084415,185.0,ALA,4.0,State,1.0,1.0,0.0,0.0,38599.895707,0.595,0.625,1
2,026ed9b585ddcd860d59a367139d7429,Bay Area 511 AC Transit Schedule,10,"LINESTRING (5377530.006 3182846.533, 5377574.4...",113,40538.084415,238.0,ALA,4.0,State,1.0,1.0,0.0,0.0,54249.258347,0.289,0.216,1


In [38]:
shs_pa['mi_on_shs'] = shs_pa.route_length * shs_pa.pct_route / geography_utils.FEET_PER_MI

In [39]:
shs_mileage = (shs_pa
 >> select(_.feed_key, _.route_id, _.mi_on_shs)
 >> group_by(_.feed_key, _.route_id)
 >> summarize(mi_on_shs = _.mi_on_shs.sum())
)

In [40]:
path = 'gs://calitp-analytics-data/data-analyses/bus_service_increase/routes_categorized_2023-04-12.parquet'

In [41]:
shs_routes = gpd.read_parquet(path)

In [42]:
shs_routes.columns

Index(['feed_key', 'name', 'route_id', 'total_routes', 'geometry', 'district',
       'service_hours', 'category', 'unique_route', 'route_length_mi'],
      dtype='object')

In [43]:
shs_routes >> head(3)

,feed_key,name,route_id,total_routes,geometry,district,service_hours,category,unique_route,route_length_mi
0,026ed9b585ddcd860d59a367139d7429,Bay Area 511 AC Transit Schedule,10,113,"LINESTRING (-122.08709 37.67010, -122.08693 37...",4.0,70.90,on_shn,1,7.68
1,026ed9b585ddcd860d59a367139d7429,Bay Area 511 AC Transit Schedule,12,113,"LINESTRING (-122.30044 37.87949, -122.30023 37...",4.0,103.55,other,1,11.30
2,026ed9b585ddcd860d59a367139d7429,Bay Area 511 AC Transit Schedule,14,113,"LINESTRING (-122.22546 37.77552, -122.22563 37...",4.0,105.67,intersects_shn,1,10.89


In [44]:
df = (shs_routes
      >> filter(-_.district.isna())
      >> inner_join(_, shs_mileage, on = ['feed_key', 'route_id'])
     )

In [45]:
df >> head(3)

,feed_key,name,route_id,total_routes,geometry,district,service_hours,category,unique_route,route_length_mi,mi_on_shs
0,026ed9b585ddcd860d59a367139d7429,Bay Area 511 AC Transit Schedule,10,113,"LINESTRING (-122.08709 37.67010, -122.08693 37...",4.0,70.90,on_shn,1,7.68,11.217072
1,026ed9b585ddcd860d59a367139d7429,Bay Area 511 AC Transit Schedule,12,113,"LINESTRING (-122.30044 37.87949, -122.30023 37...",4.0,103.55,other,1,11.30,10.020702
2,026ed9b585ddcd860d59a367139d7429,Bay Area 511 AC Transit Schedule,14,113,"LINESTRING (-122.22546 37.77552, -122.22563 37...",4.0,105.67,intersects_shn,1,10.89,11.832590


In [48]:
(df >> group_by(_.district)
    >> summarize(n = _.shape[0], mi_on_shs = _.mi_on_shs.sum())
    # >> spread('category', 'n')
).mi_on_shs.sum()

29866.570474117514

In [49]:
(df
    >> count(_.district, _.category)
    >> spread('category', 'n')
    >> mutate(total = _.intersects_shn + _.on_shn + _.other)
).sum()

district            78.0
intersects_shn    1248.0
on_shn             566.0
other              387.0
total             2185.0
dtype: float64

In [40]:
# df.to_csv('df.csv')

# Frequent SHN Routes

* quickly look for frequent transit routes using the State Highway Network
* also evaluate applicable district-sourced ideas

### CT Links:

https://app.smartsheet.com/folders/whC3J79cMJCf3MqQ7MGv8pxHgQ3JQ68VwGQGwX21

https://caltrans-my.sharepoint.com/:p:/g/personal/audrey_ogden_dot_ca_gov/EZbU16vBqYZHvrORsT-LzwsBBxJCZqfUPaKHmm5uqWmn_Q?e=4%3A4Ijf9v&CID=80D7A50B-5D1F-497F-8C92-9435B5782132&wdLOR=c3E29B53E-4F87-436E-BEDA-3F4281A53835


## pbar

In [5]:
pbar = tqdm()

0it [00:00, ?it/s]

# SFMTA

In [6]:
## parameters cell
itp_id = 282

In [7]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date, pbar)

In [8]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

{"organization_name": "City and County of San Francisco"}


In [9]:
rt_day.set_filter(route_names = ['43', '28', '29'])

In [10]:
_m = rt_day.segment_speed_map()

## Muni Lombard

In [11]:
rt_day.autocorridor(shape_id = '2856', stop_seq_range = [27, 32])

In [12]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

,route_short_name
0,43
1,49
2,91
3,22
4,90
5,30
6,28


In [13]:
rt_day.corridor_metrics()

metrics attached to self.corridor: 


,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,geometry
0,28,28,City and County of San Francisco,9.3,11.8,13.2,480.1,1690.8,56.4,159,14.9,5.3,8.5,5.6,1.1,1739.0,16,"POLYGON ((-214489.679 -21257.665, -214484.673 ..."
1,43,43,City and County of San Francisco,8.1,9.5,10.1,1210.7,1690.8,73.2,143,14.8,4.8,16.5,5.4,1.1,NaN,16,"POLYGON ((-214489.679 -21257.665, -214484.673 ..."


In [14]:
rt_day.quick_map_corridor()

In [15]:
# m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)
# m

In [16]:
all_corridors = []

In [17]:
rt_day.corridor['corridor'] = 'us101_lombard'

In [18]:
all_corridors += [rt_day.corridor]

## Muni 19th A (add, check if B worth it)

In [19]:
rt_day.autocorridor(shape_id = '2808', stop_seq_range = [21, 32])

In [20]:
# rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [21]:
rt_day.corridor_metrics()

metrics attached to self.corridor: 


,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,geometry
0,28,28,City and County of San Francisco,9.6,11.5,11.9,1061.7,1347.9,56.4,159,14.9,5.3,18.8,6.0,2.4,3850.5,16,"POLYGON ((-217949.193 -26918.889, -217955.214 ..."
1,29,29,City and County of San Francisco,8.7,11.0,12.7,286.2,1347.9,70.7,180,15.0,6.0,4.0,6.1,2.4,NaN,16,"POLYGON ((-217949.193 -26918.889, -217955.214 ..."


In [22]:
rt_day.quick_map_corridor()

In [23]:
# m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)
## geoms error, no time to fix

In [24]:
rt_day.corridor['corridor'] = 'sr1_19th_a'

In [25]:
all_corridors += [rt_day.corridor]

## Muni 19th B

In [26]:
rt_day.autocorridor(shape_id = '2808', stop_seq_range = [32, 37])

In [27]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

,route_short_name
0,58
1,91
2,18
3,28
4,M
5,57
6,23
7,29


In [28]:
rt_day.corridor_metrics()

metrics attached to self.corridor: 


,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,geometry
0,28,28,City and County of San Francisco,10.2,11.9,13.1,506.9,956.6,56.4,159,14.9,5.3,9.0,5.6,1.4,2323.2,16,"POLYGON ((-217636.048 -31045.226, -217636.036 ..."
1,29,29,City and County of San Francisco,7.9,9.9,10.3,449.7,956.6,70.7,180,15.0,6.0,6.4,6.2,1.4,NaN,16,"POLYGON ((-217636.048 -31045.226, -217636.036 ..."


In [29]:
rt_day.quick_map_corridor()

In [30]:
rt_day.corridor['corridor'] = 'sr1_19th_b'

In [31]:
all_corridors += [rt_day.corridor]

In [32]:
# m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

## GGT (skip)

# AC Transit

In [33]:
## parameters cell
itp_id = 4

In [34]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date, pbar)

In [35]:
rt_day.transit_priority_target_mph = 20

In [36]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

{"organization_name": "Alameda-Contra Costa Transit District"}


In [37]:
rt_day.set_filter(route_names = ['72', '72M', '72R'])

In [38]:
_m = rt_day.segment_speed_map()

## AC Transit San Pablo

In [39]:
rt_day.autocorridor(shape_id = 'shp-72R-15', stop_seq_range = [9, 19])

In [40]:
# rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [41]:
rt_day.corridor_metrics()

metrics attached to self.corridor: 


,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,geometry
0,72,72,Alameda-Contra Costa Transit District,10.5,12.5,12.8,1023.2,3623.2,83.1,62,14.8,2.1,12.3,2.5,6.1,9833.9,20,"POLYGON ((-201973.660 -11619.671, -201974.344 ..."
1,72M,72M,Alameda-Contra Costa Transit District,11.1,12.1,12.7,1001.4,3623.2,79.4,62,14.6,2.1,12.6,2.6,6.1,NaN,20,"POLYGON ((-201973.660 -11619.671, -201974.344 ..."
2,72R,72R,Alameda-Contra Costa Transit District,12.0,13.3,13.6,1598.6,3623.2,62.4,126,13.2,4.8,25.6,5.7,6.1,NaN,20,"POLYGON ((-201973.660 -11619.671, -201974.344 ..."


In [42]:
rt_day.quick_map_corridor()

In [43]:
# m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

In [44]:
rt_day.corridor['corridor'] = 'sr123_san_pablo'

In [45]:
all_corridors += [rt_day.corridor]

## AC Transit San Pablo rider delay

In [46]:
avg_daily_rider = 14789

In [47]:
## 15 min per trip delay

In [48]:
14789 * 15 / 60

3697.25

In [49]:
rt_day.corridor

,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,geometry,corridor
0,72,72,Alameda-Contra Costa Transit District,10.5,12.5,12.8,1023.2,3623.2,83.1,62,14.8,2.1,12.3,2.5,6.1,9833.9,20,"POLYGON ((-201973.660 -11619.671, -201974.344 ...",sr123_san_pablo
1,72M,72M,Alameda-Contra Costa Transit District,11.1,12.1,12.7,1001.4,3623.2,79.4,62,14.6,2.1,12.6,2.6,6.1,NaN,20,"POLYGON ((-201973.660 -11619.671, -201974.344 ...",sr123_san_pablo
2,72R,72R,Alameda-Contra Costa Transit District,12.0,13.3,13.6,1598.6,3623.2,62.4,126,13.2,4.8,25.6,5.7,6.1,NaN,20,"POLYGON ((-201973.660 -11619.671, -201974.344 ...",sr123_san_pablo


# BBB (add)

In [50]:
## parameters cell
itp_id = 300

In [51]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date, pbar)

In [52]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

{"organization_name": "City of Santa Monica"}


In [53]:
rt_day.set_filter(route_names = ['3', 'R3'])

In [54]:
_m = rt_day.segment_speed_map()

In [55]:
# _m

## BBB Lincoln

In [56]:
rt_day.autocorridor(shape_id = '26355', stop_seq_range = [16, 29])

In [57]:
# rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [58]:
rt_day.corridor_metrics()

metrics attached to self.corridor: 


,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,geometry
0,3606,3,City of Santa Monica,10.5,13.1,12.8,832.6,925.9,52.5,111,14.8,3.8,15.9,4.3,4.1,6574.5,16,"POLYGON ((143584.535 -446424.841, 143543.165 -..."
1,3620,R3,City of Santa Monica,6.1,6.4,7.1,93.4,925.9,83.9,4,9.0,0.2,1.1,0.3,4.1,NaN,16,"POLYGON ((143584.535 -446424.841, 143543.165 -..."


In [59]:
rt_day.quick_map_corridor()

In [60]:
# m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

In [61]:
rt_day.corridor['corridor'] = 'sr1_lincoln'

In [62]:
all_corridors += [rt_day.corridor]

# LA Metro

In [63]:
## parameters cell
itp_id = 182

In [64]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date, pbar)

In [65]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

{"organization_name": "Los Angeles County Metropolitan Transportation Authority"}


In [66]:
rt_day.set_filter(route_names = ['4'])

In [67]:
_m = rt_day.segment_speed_map()

## Metro Santa Monica Bl

In [68]:
rt_day.autocorridor(shape_id = '40336_DEC22', stop_seq_range = [39, 49])

In [69]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

,route_short_name
0,224
1,210
2,207
3,4
4,212


In [70]:
rt_day.corridor_metrics()

metrics attached to self.corridor: 


,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,geometry
0,4-13167,4,Los Angeles County Metropolitan Transportation...,7.8,9.3,9.8,1907.2,1907.2,117.0,243,14.7,8.2,16.3,8.8,2.0,3237.3,16,"POLYGON ((152799.930 -434869.235, 152799.237 -..."


In [71]:
rt_day.quick_map_corridor()

In [72]:
# m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

In [73]:
rt_day.corridor['corridor'] = 'sr2_santa_monica_bl'

In [74]:
all_corridors += [rt_day.corridor]

# VTA (add, optional)

In [75]:
## parameters cell
itp_id = 294

In [76]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date, pbar)

In [77]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

{"organization_name": "Santa Clara Valley Transportation Authority"}


In [78]:
rt_day.set_filter(route_names = ['22', 'Rapid 522'])

In [79]:
_m = rt_day.segment_speed_map()

In [80]:
# _m

## VTA ECR

In [81]:
rt_day.autocorridor(shape_id = '108676', stop_seq_range = [41, 98])

In [82]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

,route_short_name
0,51H
1,51
2,53
3,21
4,56
5,40
6,60
7,22
8,55
9,52


In [83]:
rt_day.corridor_metrics()

metrics attached to self.corridor: 


,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,geometry
0,22,22,Santa Clara Valley Transportation Authority,11.4,12.7,13.2,2541.8,2630.6,124.3,126,14.8,4.3,20.4,4.9,15.7,25196.6,16,"POLYGON ((-185666.588 -67116.893, -185783.755 ..."
1,Rapid 522,Rapid 522,Santa Clara Valley Transportation Authority,14.5,15.9,17.0,88.8,2630.6,98.1,114,14.8,3.9,0.9,3.9,15.7,NaN,16,"POLYGON ((-185666.588 -67116.893, -185783.755 ..."


In [84]:
rt_day.quick_map_corridor()

In [85]:
rt_day.corridor['corridor'] = 'sr82_vta'

In [86]:
all_corridors += [rt_day.corridor]

In [87]:
# m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

# Samtrans (add, optional)

In [88]:
## parameters cell
itp_id = 290

In [89]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date, pbar)

In [90]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

{"organization_name": "San Mateo County Transit District"}


In [91]:
rt_day.set_filter(route_names = ['ECR'])

In [92]:
_m = rt_day.segment_speed_map()

## Samtrans ECR

merge shape, don't do 2 parts...

In [93]:
rt_day.autocorridor(shape_id = 'ECR1095', stop_seq_range = [3, 65])

In [94]:
corr_copy = rt_day.corridor.copy()

In [95]:
rt_day.autocorridor(shape_id = 'ECR1098', stop_seq_range = [4, 35])

In [96]:
corr2_copy = rt_day.corridor.copy()

In [97]:
gdf = pd.concat([corr_copy, corr2_copy])

In [98]:
gdf.distance_meters = gdf.distance_meters.sum()
gdf = gdf.dissolve()

In [99]:
# gdf.explore()

In [100]:
rt_day.add_corridor(gdf)

In [101]:
# rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [102]:
rt_day.corridor_metrics()

metrics attached to self.corridor: 


,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,geometry
0,ECR,ECR,San Mateo County Transit District,11.8,12.8,13.5,3009.7,3009.7,126.4,129,14.9,4.3,23.8,5.1,25.4,40858.2,16,"POLYGON ((-205312.556 -47845.139, -205315.667 ..."


In [103]:
rt_day.quick_map_corridor()

In [104]:
# m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

# m

In [105]:
rt_day.corridor['corridor'] = 'sr82_samtrans'

In [106]:
all_corridors += [rt_day.corridor]

# Foothill (eval)

In [107]:
## parameters cell
itp_id = 112

In [108]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date, pbar)

In [109]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

{"organization_name": "Foothill Transit"}


In [110]:
rt_day.set_filter(route_names = ['188'])

In [111]:
_m = rt_day.segment_speed_map()

## Foothill Foothill Bl

In [112]:
rt_day.autocorridor(shape_id = '12286_shp', stop_seq_range = [1779, 2834])

In [113]:
# rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [114]:
rt_day.corridor_metrics()

metrics attached to self.corridor: 


,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,geometry
0,188,188,Foothill Transit,12.3,15.2,17.5,154.6,154.6,66.5,94,14.9,3.2,2.3,3.2,5.4,8725.7,16,"POLYGON ((203832.442 -430395.169, 203827.149 -..."


In [115]:
rt_day.quick_map_corridor()

In [116]:
rt_day.corridor['corridor'] = 'sr66_foothill'

In [117]:
all_corridors += [rt_day.corridor]

# From Project List
https://app.smartsheet.com/reports/4gRrr282hJ3MPVG7hcqX4XQHh2FR4GgpfW6pq5Q1?view=grid

In [ ]:
# rt_utils.show_full_df((pd.read_parquet('./_rt_progress_2023-05-17.parquet')
#                                    >> filter(_.status == 'map_confirmed')
#                                    )
#                                   )

In [ ]:
# analysis_date = dt.date(2023, 5, 17)

# Eureka

In [118]:
## parameters cell
itp_id = 108

In [119]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date, pbar)

In [120]:
rt_day.transit_priority_target_mph = 20

In [121]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

{"organization_name": "City of Eureka"}


In [122]:
# rt_day.set_filter(route_names = ['188'])

In [123]:
try:
    _m = rt_day.segment_speed_map()
except:
    pass

## Eureka US101 (Broadway Truesdale - 4th per [RTP](https://www.hcaog.net/sites/default/files/vroom_2022-2042_full_report.pdf))

* difficult to analyze -- need to exclude distant stops from corridor (add to algorithm?)
* standardize on most common trip pattern

In [124]:
rt_day.autocorridor(shape_id = 'p_178050', stop_seq_range = [13, 13.5],
manual_exclude={'p_179049': {'max': 4}, 'p_179044': {'max': 4}})
# intercity service, exclude stops distant from corridor

In [125]:
# rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [126]:
rt_day.corridor_metrics()

metrics attached to self.corridor: 


,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,geometry
0,1,RTS,City of Eureka,9.5,10.8,11.2,153.5,165.4,115.8,30,12.4,1.2,1.3,1.3,0.8,1283.1,20,"POLYGON ((-353644.983 314814.857, -353646.504 ..."
1,3,NaN,City of Eureka,13.5,13.7,13.7,5.1,165.4,50.8,2,3.0,0.3,0.1,0.4,0.8,NaN,20,"POLYGON ((-353644.983 314814.857, -353646.504 ..."
2,33023,RTS,City of Eureka,12.4,22.3,21.3,-32.8,165.4,44.0,21,9.9,1.1,-0.7,1.0,0.8,NaN,20,"POLYGON ((-353644.983 314814.857, -353646.504 ..."
3,4,NaN,City of Eureka,11.7,12.3,12.3,14.9,165.4,51.7,9,9.0,0.5,0.3,0.5,0.8,NaN,20,"POLYGON ((-353644.983 314814.857, -353646.504 ..."
4,5,NaN,City of Eureka,8.3,9.0,8.7,22.3,165.4,50.0,5,10.0,0.3,0.4,0.3,0.8,NaN,20,"POLYGON ((-353644.983 314814.857, -353646.504 ..."
5,822,NaN,City of Eureka,13.7,15.1,15.1,2.5,165.4,108.8,5,10.5,0.2,0.0,0.2,0.8,NaN,20,"POLYGON ((-353644.983 314814.857, -353646.504 ..."


In [127]:
rt_day.quick_map_corridor()

In [128]:
rt_day.corridor['corridor'] = 'us101_broadway'

In [129]:
all_corridors += [rt_day.corridor]

# TART SR267/89 -- no data

# Tahoe Transportation District US50

* limited data geographic extent

In [130]:
## parameters cell
itp_id = 331

In [131]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date, pbar)

In [132]:
rt_day.transit_priority_target_mph = 25

In [133]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

{"organization_name": "Tahoe Transportation District"}


In [134]:
_m = rt_day.segment_speed_map()

In [135]:
rt_day.autocorridor(shape_id = 'p_497358', stop_seq_range = [0, 20])

In [136]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

,route_short_name
0,22
1,50
2,55
3,19X


In [137]:
rt_day.corridor_metrics()

metrics attached to self.corridor: 


,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,geometry
0,12133,22,Tahoe Transportation District,11.1,11.1,11.1,0.8,417.1,32.7,7,12.0,0.3,0.0,0.3,4.6,7443.8,25,"POLYGON ((-493.861 99756.139, -489.943 99765.0..."
1,12134,50,Tahoe Transportation District,15.8,18.2,17.8,181.6,417.1,17.3,20,13.1,0.8,10.5,1.2,4.6,NaN,25,"POLYGON ((-493.861 99756.139, -489.943 99765.0..."
2,12135,19X,Tahoe Transportation District,1.9,1.9,1.9,7.3,417.1,34.4,8,11.9,0.3,0.2,0.3,4.6,NaN,25,"POLYGON ((-493.861 99756.139, -489.943 99765.0..."
3,12137,55,Tahoe Transportation District,12.3,15.6,27.3,227.4,417.1,41.7,17,11.2,0.8,5.5,1.0,4.6,NaN,25,"POLYGON ((-493.861 99756.139, -489.943 99765.0..."


In [138]:
rt_day.quick_map_corridor()

In [139]:
rt_day.corridor['corridor'] = 'us50_tahoe'

In [140]:
all_corridors += [rt_day.corridor]

# Fresno -- unable eval (new services)

# VCTC (SR126 Fillmore - Ventura)

* minimal thru running delay on available data...

In [ ]:
## parameters cell
itp_id = 380

In [ ]:
# analysis_date = dt.date(2023, 4, 12)

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date, pbar)

In [ ]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

In [ ]:
_m = rt_day.segment_speed_map()

In [ ]:
_m

# LA Metro Projects

* NoHo-Pas BRT to only briefly use SR134, light delay
* As per sheet, NSFV improvements only intersect SHN
* Inglewood connector, Arbor Vitae not on SHN...
* Games Route Network, etc unable to analyze

# Victor Valley

* service runs, difficult to evaluate
* 0.5/hour, limited span

# Others

* OCTA speculative, new services unable to eval

# OCTA SR39

* multiple SHN sections, eval longest
* about 5 trips/hour combined!

In [141]:
## parameters cell
itp_id = 235

In [142]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date, pbar)

In [143]:
rt_day.transit_priority_target_mph = 20

In [144]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

{"organization_name": "Orange County Transportation Authority"}


In [145]:
rt_day.set_filter(route_names=['29', '529'])

In [146]:
_m = rt_day.segment_speed_map()

In [147]:
rt_day.autocorridor(shape_id = '2902', stop_seq_range = [5, 51])

In [148]:
# rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [149]:
rt_day.corridor_metrics()

metrics attached to self.corridor: 


,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,geometry
0,29,29,Orange County Transportation Authority,11.5,13.1,13.6,1905.7,2211.5,92.6,75,14.8,2.5,20.6,3.2,12.7,20375.8,20,"POLYGON ((185851.989 -468884.655, 185851.980 -..."
1,529,529,Orange County Transportation Authority,14.6,16.1,16.6,305.8,2211.5,38.6,56,12.2,2.3,7.9,2.6,12.7,NaN,20,"POLYGON ((185851.989 -468884.655, 185851.980 -..."


In [150]:
rt_day.quick_map_corridor()

In [151]:
rt_day.corridor['corridor'] = 'sr39_beach'

In [152]:
all_corridors += [rt_day.corridor]

# combine and export

In [167]:
all_gdf = pd.concat(all_corridors)

In [168]:
all_gdf.columns

Index(['route_id', 'route_short_name', 'organization', 'p20_corr_mph',
       'p50_corr_mph', 'avg_corr_mph', 'speed_delay_minutes',
       'total_speed_delay', 'p50_runtime_minutes', 'n_trips', 'span_hours',
       'daily_avg_trips_hr', 'trips_added', 'new_avg_trips_hr', 'length_miles',
       'distance_meters', 'target_mph', 'geometry', 'corridor'],
      dtype='object')

In [ ]:
# (all_gdf >> select(-_.geometry)).to_csv('june16_corridor_metrics_speeds.csv')

In [155]:
all_gdf.to_file('june16_corridor_metrics_speeds.geojson')

In [156]:
all_gdf = gpd.read_file('./june16_corridor_metrics_speeds.geojson')

In [52]:
all_gdf >> filter(_.corridor == 'sr123_san_pablo')

,route_id,route_short_name,organization,p20_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,target_mph,corridor,geometry
6,72,72,Alameda-Contra Costa Transit District,10.5,1023.2,3623.2,83.1,62,14.8,2.1,12.3,2.5,6.1,20,sr123_san_pablo,"POLYGON ((-201973.660 -11619.671, -201974.344 ..."
7,72M,72M,Alameda-Contra Costa Transit District,11.1,1001.4,3623.2,79.4,62,14.6,2.1,12.6,2.6,6.1,20,sr123_san_pablo,"POLYGON ((-201973.660 -11619.671, -201974.344 ..."
8,72R,72R,Alameda-Contra Costa Transit District,12.0,1598.6,3623.2,62.4,126,13.2,4.8,25.6,5.7,6.1,20,sr123_san_pablo,"POLYGON ((-201973.660 -11619.671, -201974.344 ..."


In [157]:
all_gdf = (all_gdf >> group_by(_.corridor, _.organization)
    >> mutate(avg_trip_delay_all_rts = _.total_speed_delay / _.n_trips.sum())
    >> summarize(avg_delay_min = _.avg_trip_delay_all_rts.max())
)

In [158]:
all_gdf

,corridor,organization,avg_delay_min
0,sr123_san_pablo,Alameda-Contra Costa Transit District,14.492800
1,sr1_19th_a,City and County of San Francisco,3.976106
2,sr1_19th_b,City and County of San Francisco,2.821829
3,sr1_lincoln,City of Santa Monica,8.051304
4,sr2_santa_monica_bl,Los Angeles County Metropolitan Transportation...,7.848560
5,sr39_beach,Orange County Transportation Authority,16.881679
6,sr66_foothill,Foothill Transit,1.644681
7,sr82_samtrans,San Mateo County Transit District,23.331008
8,sr82_vta,Santa Clara Valley Transportation Authority,10.960833
9,us101_broadway,City of Eureka,2.297222


In [159]:
corrs = all_gdf.corridor.to_list()
corrs

['sr123_san_pablo',
 'sr1_19th_a',
 'sr1_19th_b',
 'sr1_lincoln',
 'sr2_santa_monica_bl',
 'sr39_beach',
 'sr66_foothill',
 'sr82_samtrans',
 'sr82_vta',
 'us101_broadway',
 'us101_lombard',
 'us50_tahoe']

In [160]:
import numpy as np

In [161]:
ridership = [14789, 12100, 12100, #19th b
            6200, 22669, 4912, #beach
            5681, 8754, 15693, #vta
            np.nan, 8200, np.nan]

In [162]:
rider = pd.DataFrame({'corridor': corrs, 'avg_daily_riders': ridership})

In [165]:
df = (all_gdf >> inner_join(_, rider, on = 'corridor')
 >> mutate(rider_minutes = _.avg_delay_min * _.avg_daily_riders,
                rider_hours = _.rider_minutes / 60)
 >> arrange(-_.rider_hours)
)
df.loc[0,'corridor'] = 'sr123_san_pablo_full'
df.loc[4,'corridor'] = 'sr2_hollywood'
df['submission_round'] = 1
df

,corridor,organization,avg_delay_min,avg_daily_riders,rider_minutes,rider_hours,submission_round
0,sr123_san_pablo_full,Alameda-Contra Costa Transit District,14.492800,14789.0,214334.019200,3572.233653,1
7,sr82_samtrans,San Mateo County Transit District,23.331008,8754.0,204239.641860,3403.994031,1
4,sr2_hollywood,Los Angeles County Metropolitan Transportation...,7.848560,22669.0,177918.999177,2965.316653,1
8,sr82_vta,Santa Clara Valley Transportation Authority,10.960833,15693.0,172008.357500,2866.805958,1
5,sr39_beach,Orange County Transportation Authority,16.881679,4912.0,82922.809160,1382.046819,1
3,sr1_lincoln,City of Santa Monica,8.051304,6200.0,49918.086957,831.968116,1
1,sr1_19th_a,City and County of San Francisco,3.976106,12100.0,48110.884956,801.848083,1
10,us101_lombard,City and County of San Francisco,5.598675,8200.0,45909.139073,765.152318,1
2,sr1_19th_b,City and County of San Francisco,2.821829,12100.0,34144.129794,569.068830,1
6,sr66_foothill,Foothill Transit,1.644681,5681.0,9343.431915,155.723865,1
